<h1>Analisi della densita' e della sparsita' di un dataset</h1>

<h2>Caricamento del dataframe</h2>

Di seguito effettuo l'analisi di un modello, basato sul dataset `Goodreads-10K Dataset`. Per farlo utilizzo nozioni note per i sistemi di raccomandazione.

I tasks del mio lavoro si suddividono in:
- **Caricamento dei dati:** con lettura del dataset e creazione dei dataframe di lavoro;
- **Caratteristiche dei dati:** in questa parte, ho ritenuto utile utilizzare metodi della classe `pandas.DataFrame`;
- **Visualizzazione dei dati:** con la rappresentazione dei dati su istogrammi. Ho deciso di valutare il rapporto tra numero medio di valutazioni utente e valore di appartenenza.
- **Analisi dei dati:** con cui ho approfondito l'informazione contenuta all'interno dei datasets. Per farlo mi sono costruita del codice "query";
- **Calcolo della densita' e sparsita' della matrice di rating:** con cui ho valutato la matrice di rating.

Il dataset `Goodreads-10K Dataset` contiene tre cvs: Books, Ratings e to_read.

Io ho deciso di concentrarmi su:
- `Books.csv` e di estrapolarne solo le informazioni indispensabili: *titolo* del libro con alcuni metadati (book id, anno di pubblicazione e *autore*), evitando per esempio il codice ISBN non rilevante ai fini del mio lavoro, e la *valutazione media degli utenti* data a un certo libro.
- `Ratings.csv`: con *id del libro*, *id dell'utente* e *voto dato dall'utente*;
- `to_read.csv`: con *id del libro* e *id dell'utente*.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

book = pd.read_csv('../input/goodreads10k-dataset-cleaned/Books.csv', usecols=["book_id", "Title", "Author","Publication Year","AvgRating"], encoding='utf-8') 
book.head(10) # stampa delle prime 10 righe del dataframe

Il dataframe `book` l'ho usato per la visualizzazione delle relazioni titolo libro-votazione utente media. L'ho considerato il dataframe principale di lavoro, ed anche quello su cui ho svolto l'implementazione del codice "query".

In [ ]:
rating = pd.read_csv('../input/goodreads10k-dataset-cleaned/Ratings.csv', encoding='utf-8') 
rating.head(10) # stampa delle prime 10 righe del dataframe

In [ ]:
to_read = pd.read_csv('../input/goodreads10k-dataset-cleaned/to_read.csv', encoding='utf-8') 
to_read.head(10) # stampa delle prime 10 righe del dataframe

I dataframes `rating` e `to_read` gli ho usati per calcolarmi la matrice di rating, e di conseguenza per il calcolo di densita' e sparsita' della stessa.

<h2>Caratteristiche dei dati</h2>

**shape** e' un campo proprio del dataframe `book`, e mi ha permesso di comprenderne le dimensioni: 10 000 campioni e 5 features.

In [ ]:
book.shape

Il metodo **info()** mi ha permesso di visualizzare un sommario conciso del dataframe in esame. Da come si puo' vedere sotto, mi sono accorta di avere: per ogni features 10 000 valori non nulli; di tipo int64 per le features numeriche a valori interi; di tipo object per le features di stringhe; e di tipo flot64 per la valutazione utente (che e' in virgola mobile).

In [ ]:
book.info()

Il metodo **describe()** mi ha permesso di generare statistiche descrittive. Per una maggiore comprensione dei risultati che ho raccolto:
- count, conta il numero di osservazioni non NA / nulle;
- mean, la media dei valori;
- std, la deviazione standard delle osservazioni;
- min, il minimo dei valori nella feature;
- 25%, 50%, 75%, sono alcune stime medie sui valori. Da notarsi come il 50% indica la mediana;
- max, il massimo dei valori nella feature.

La media delle valutazioni ottenute da tutti i libri sono attorno al 4.00, la mediana attorno al 4.02, in un range tra 2.47 e 4.82.

In [ ]:
book.describe()

<h2>Visualizzazione dei dati</h2>

Una volta creati i dataframe di lavoro, comprese le dimensioni e i valori che vi sono contenuti; sono passata alla creazione di un plot che distribuisse in classi di punteggi, le valutazioni medie ottenute dagli utenti.

Per costruirlo ho usato il dataframe `book`.

In [ ]:
# Rappresento la disribuzione dei libri per valutazione utente media

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif']=['SimHei'] # permette di compilare caratteri cinesi

fig, axs = plt.subplots(1,1,figsize=(20,30)) 

sns.countplot(y  = "AvgRating", data = book, palette="Set3")
axs.set_title('Distribuzione valutazione media utenti', fontweight='bold')
plt.show()

Come gia' preanunciato, dal valore della media e della mediana, il maggior numero di valutazioni (si vede in figura) e' attorno a [3.91;4.16], con un numero di conteggi anche superiore a 175. Invece agli estremi la situzione decresce molto rapidamente, fino a diventare 0 in > 4.82 e < 2.47 (valutazioni di conseguenza mai date da nessun utente).

Vista la distribuzione dei valori, posso assumere che le valutazioni > 3.9, significano che il libro ha avuto un riscontro, in media, positivo negli utenti.

<h2>Analisi dei dati</h2>

L'analisi dei dati l'ho svolta nel dataframe `book`.

<h3> Codice "query"</h3>

Il codice "query" che ho creato estrapola le seguenti informazioni:

* Gli autori per indice di gradimento complessivo, piu' alto (rating per autore): `query_rating_complessivo_per_autore`

In [ ]:
# Ordino gli autori per indice di gradimento complessivo, piu' alto (rating per autore)
def query_rating_complessivo_per_autore():
    best_author = book.groupby('Author').sum() # raggruppo il dataframe per Author. Su ogni caratteristica faccio la somma dei valori
    best_author.sort_values(by=['AvgRating'], ascending = False, inplace = True) # ascending rating in ordine decrescente, inplace la modifica e' applicata al dataframe originale
    best_author = best_author['AvgRating'] 
    print("Classificazione degli autori per votazione media complessiva in ordine decrescente (primi 10): \n", best_author.head(10)) # i primi 10 autori con rating piu' elevato
    return best_author

* I titoli che hanno ottenuto una votazione media utente maggiore: `query_libri_con_ratingsingolo_maggiore`

In [ ]:
# Ordino i titoli sulla media del voto ottenuto da ogni titolo (rating per libro) e ne seleziono i 10 titoli con voto medio maggiore
def query_libri_con_ratingsingolo_maggiore():

# Ordino i titoli sulla media del voto ottenuto da ogni titolo (rating per libro)
# Coppie lette per ogni libro
    copy  = book.groupby(['Author', 'Title']).mean() # raggruppo i libri per Author e per Title. Su ogni caratteristica calcolo la media dei valori.
    print("Statistiche descrittive raggruppate per autore e libro: ")
    print(copy.describe())
    print("\n")
# Ogni libro ha solo una copia

# Seleziono i 10 titoli con voto medio maggiore
    copy.sort_values(by = 'AvgRating', ascending = False, inplace = True)  # ordino i rating in modo decresecente
    best_book = copy['AvgRating']
    print("Classificazione dei autori-titoli per votazione media in ordine decrescente (primi 10): \n", best_book.head(10))
    return best_book



* I primi 3 autori con voto complessivo maggiore, che hanno anche un libro classificato tra i primi 10 piu' votati: `query_3autori_con_votocomplessivomaggiore_e_librosingoloclassificatoprimi10()`

Ho ritenuto opportuno anche implementare 3 sottquery alla query principale:
- `sottoquery_migliori_autori_ratingprimi10titoli`
- `sottoquery_migliori_autori_ratingcomplessivoperautore`
- `sottoquery_merge_migliori_autori_ratingprimi10titoli_with_migliori_autori_ratingcomplessivoperautore` con  un metodo di supporto `no_duplicate`

In [ ]:
# Metodo di supporto, ritorna false quando nella lista l e' gia' contenuto l'elemento value
def no_duplicate(l, value):
    for i in l:
        if(value == i):
            return False
    return True

In [ ]:
def sottoquery_migliori_autori_ratingprimi10titoli(best_book):
    ten_bbook = best_book[:10] # prendo i primi 10 rating dei libri migliori (da best_book)

# e di questi 10 libri migliori estraggo i loro autori, in ordine di rating decrescente (da book)
    ten_best_author_avg = []
    for i in ten_bbook:
# cerco i titoli a cui e' assegnato ciascun rating, e di questi ne seleziono l'autore
        row = 0
        for j in book['AvgRating']:
            if i == j:
                ten_best_author_avg.append(book.loc[row]['Author'])
                break
            row = row + 1
        
    print("I migliori autori, considerando il miglior rating dei primi 10 titoli: ")
    print(list(set(ten_best_author_avg))) # tolgo i duplicati  
    return ten_best_author_avg

In [ ]:
# Estraggo da best_author (la somma del punteggio per ogni autore) il nome del autore
def sottoquery_migliori_autori_ratingcomplessivoperautore(best_author):
    best_author_sum = []
    best_author_sum_axes = best_author.axes # prendo la label
    element = best_author_sum_axes[0].values
    for i in element:
        best_author_sum.append(i)
        
    print("\n")
    print("I migliori autori, considerando il rating complessivo per autore: ")
    print(list(set(best_author_sum)))
    return best_author_sum

In [ ]:
# Cerco su best_author_sum se gli autori sono presenti anche in ten_best_book. Estraggo i primi 3
def sottoquery_merge_migliori_autori_ratingprimi10titoli_with_migliori_autori_ratingcomplessivoperautore(ten_best_author_avg, best_author_sum):
    three_best_author = []
    total_best_author = []
    n_authors = 0
    for i in best_author_sum:
        for j in ten_best_author_avg:
            if i == j:
                if(no_duplicate(total_best_author, i)):
                        total_best_author.append(i)
                        n_authors = n_authors + 1
                        if(n_authors <= 3):
                            three_best_author.append(i)
    print("\n")
    print("I migliori 3 autori nei primi 10 libri ratati sono: ")
    print(three_best_author) 
    return total_best_author

In [ ]:
# Estraggo i primi 3 autori con rating maggiore, che hanno anche un libro classificato tra i primi 10 rating. 
# In caso di piu' libri, prendo quello con rating maggiore.

# Ritorno il caso piu' generale (gli autori con rating maggiore, che hanno anche un libro classificato tra i primi 10 rating)

def query_3autori_con_votocomplessivomaggiore_e_librosingoloclassificatoprimi10(best_book, best_author):
    ten_best_author_avg = sottoquery_migliori_autori_ratingprimi10titoli(best_book)
    best_author_sum = sottoquery_migliori_autori_ratingcomplessivoperautore(best_author)
    return sottoquery_merge_migliori_autori_ratingprimi10titoli_with_migliori_autori_ratingcomplessivoperautore(ten_best_author_avg, best_author_sum)

<h3>Main</h3>

Il main che manda in esecuzione le 3 query principali e' il seguente:

In [ ]:
### MAIN ###
best_author = query_rating_complessivo_per_autore()

In [ ]:
best_book = query_libri_con_ratingsingolo_maggiore()

In [ ]:
total_best_author = query_3autori_con_votocomplessivomaggiore_e_librosingoloclassificatoprimi10(best_book, best_author)

Di seguito ho voluto inserire un plot che mostrasse i migliori autori, con valutazione media migliore, dei primi 10 libri miglior ratati. Ovvero la lista di ritorno di `query_3autori_con_votocomplessivomaggiore_e_librosingoloclassificatoprimi10`.

In [ ]:
# Voglio visualizzare un grafico con total_best_author (i 10 migliori autori per rating per libro e per autore) rapportati al loro rating complessivo

totalrating_for_author = pd.DataFrame(columns=['Author', 'AvgRating'])
for name_author in total_best_author:
    totalrating_for_author.loc[name_author] = [name_author, best_author.loc[name_author]]

print("I migliori autori nei primi 10 libri ratati sono: ")
print(total_best_author)


In [ ]:
fig, ax = plt.subplots(figsize =(16, 9)) 
author = totalrating_for_author['Author']
rating_avg = totalrating_for_author['AvgRating'] 
  

my_colors = 'pink', 'orange', 'blue', 'black', 'purple' 

ax.barh(author, rating_avg, color = my_colors) 
ax.invert_yaxis() # inverto gli assi
ax.set_title('Autori con il rating complessivo maggiore nei primi 10 libri preferiti dagli utenti', fontweight='bold')   
plt.show() 

<h2>Calcolo della densita' e della sparsita' della matrice di rating</h2>

<h3>Densita'</h3>

Matrice densa significa che tutte le sue entry hanno ricevuto una votazione da parte degli utenti.

In [ ]:
# Devo capire quanti utenti non hanno dato alcuna votazione

n_users_read = to_read['user_id'].nunique() # numero di utenti che hanno letto un libro
n_users = rating['user_id'].nunique() # il numero di utenti che hanno votato
n_book = book['book_id'].nunique() # il numero di libri presenti nel dataset
n_votes = (rating['user_id'].shape)[0] # numero di voti totali

print("Numero di utenti che hanno letto un libro: ", n_users_read)
print("Numero di utenti che hanno votato (numero di users): ",n_users)
print("Numero di libri: ", n_book)
print("Numero di voti complessivi: ",n_votes)

Per calcolare il numero di utenti che, nella matrice di rating, non hanno dato alcuna informazione di preferenza su un libro, ho deciso di considerare il numero di voti complessivo, diviso il numero di utenti per il numero di libri.

Non ho vincolato il voto solo agli utenti che hanno letto un libro, in quanto gli utenti che hanno letto un libro sono in numero inferiore rispetto agli utenti che hanno espresso una preferenza.

In [ ]:
densita =  n_votes/(n_book * n_users)

In [ ]:


print("La densita' della matrice e': ", densita * 100, "%")
    

Dunque ho circa 1.8 entry su 1000, della matrice di rating, che sono diverse da 0.

<h3>Sparsita'</h3>

La sparsita' della matrice, non e' altro che il numero di 0 di cui e' composta. Applicata al mio caso significa

In [ ]:
sparsita = 1.0 - densita

print("La sparsita' della matrice e': ", sparsita*100, "%")

Ovvero piu' del 99% degli utenti non ha espresso alcuna preferenza in almeno un titolo.

<h3>Effetto Long Tail</h3>

L'effetto **Long Tail** si ha quando pochi utenti (users) hanno interagito con un numero elevato di items (in questo caso i libri); e la stragrande maggioranza invece non ha espresso alcuna preferenza. Questo e' quello che capita nel caso in esame, che gia' si evince  dalla sparsita' della matrice di rating, confermato anche dal grafico sottostante.

In [ ]:
distribution_votes = rating.groupby('user_id').count()

plt.figure(figsize=(15, 10))
plt.hist(distribution_votes["book_id"], color = 'purple')
plt.gca().set_xlabel("user_id")
plt.gca().set_ylabel("numero di voti")
plt.title('Distribuzione dei voti per users', fontweight='bold')
plt.show()